In [1]:
import os
import torch
import cv2
import numpy as np
from PIL import Image
from torchvision import transforms
from fcos import FCOSDetector
from VOC_dataset import VOCDataset

In [5]:
torch.cuda.is_available()

True

In [2]:
root_dir = './test_images/'
image_list = [os.path.join(root_dir, fn) for fn in os.listdir(root_dir) if fn.endswith('.jpg')]

In [3]:
ckpt = torch.load("./models/voc2012_512x800_epoch100_loss0.6055.pth",map_location=torch.device('cpu'))
model=FCOSDetector(mode="inference")
model.load_state_dict(ckpt)
model.eval()
print('')

INFO===>success frozen BN
INFO===>success frozen backbone stage1



In [107]:
resize = transforms.Resize((512))
to_tensor = transforms.ToTensor()
to_normalize = transforms.Normalize((0.485,0.456,0.406), (0.229,0.224,0.225))

for f_path in image_list:
    image = Image.open(image_list[idx]).convert('RGB')
    run_demo(image, model)

KeyboardInterrupt: 

In [4]:
resize = transforms.Resize((512))
to_tensor = transforms.ToTensor()
to_normalize = transforms.Normalize((0.485,0.456,0.406), (0.229,0.224,0.225))

idx = 0
image = Image.open(image_list[idx]).convert('RGB')

def run_demo(image, model):
    tensor_img = to_normalize(to_tensor(resize(image)))
    tensor_img = tensor_img.unsqueeze(0)

    image = image.resize((tensor_img.shape[3], tensor_img.shape[2]))
    cv_img = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)

    with torch.no_grad():
        scores, classes, boxes = model(tensor_img)

    boxes=boxes[0].cpu().numpy().tolist()
    classes=classes[0].cpu().numpy().tolist()
    scores=scores[0].cpu().numpy().tolist()

    for i, box in enumerate(boxes):
        pt1 = (int(box[0]), int(box[1]))
        pt2 = (int(box[2]), int(box[3]))
        cv2.rectangle(cv_img,pt1,pt2,(0,255,0))
        cv2.putText(cv_img,"%s %.3f"%(VOCDataset.CLASSES_NAME[int(classes[i])],scores[i]),(int(box[0]+5),int(box[1])+15),cv2.FONT_HERSHEY_SIMPLEX,0.5,[0,200,20],2)

    cv2.imwrite("./out_images/"+image_list[idx].split('/')[-1],cv_img)
    # cv2.imshow('', cv_img)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()